In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv("/content/drive/MyDrive/ai lab 5/IMDB Dataset.csv")

In [5]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
data.shape

(50000, 2)

In [7]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [8]:
data["sentiment"].value_counts

<bound method IndexOpsMixin.value_counts of 0        positive
1        positive
2        positive
3        negative
4        positive
           ...   
49995    positive
49996    negative
49997    negative
49998    negative
49999    negative
Name: sentiment, Length: 50000, dtype: object>

# One Hot Encoding

## Label Encoder

In [9]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

In [10]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


# Data Preprocessing

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [12]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [13]:
train_data.shape

(40000, 2)

In [14]:
test_data.shape

(10000, 2)

In [15]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data['review'])

In [16]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen = 200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen = 200)

In [17]:
X_train

array([[1935,    1, 1200, ...,  205,  351, 3856],
       [   3, 1651,  595, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]], dtype=int32)

In [18]:
X_test

array([[   0,    0,    0, ...,  995,  719,  155],
       [  12,  162,   59, ...,  380,    7,    7],
       [   0,    0,    0, ...,   50, 1088,   96],
       ...,
       [   0,    0,    0, ...,  125,  200, 3241],
       [   0,    0,    0, ..., 1066,    1, 2305],
       [   0,    0,    0, ...,    1,  332,   27]], dtype=int32)

In [19]:
Y_train = train_data['sentiment']
Y_test = test_data['sentiment']

In [20]:
Y_train

,sentiment
39087,0
30893,0
45278,1
16398,0
13653,0
...,...
11284,1
44732,1
38158,0
860,1


In [21]:
Y_test

,sentiment
33553,1
9427,1
199,0
12447,1
39489,0
...,...
28567,0
25079,1
18707,1
15200,0


# Model Building

In [22]:
model = Sequential()
model.add(Embedding(input_dim = 5000, output_dim = 128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [23]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [24]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [25]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split = 0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 223s 430ms/step - accuracy: 0.7231 - loss: 0.5243 - val_accuracy: 0.8493 - val_loss: 0.3522
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 249s 414ms/step - accuracy: 0.8569 - loss: 0.3467 - val_accuracy: 0.7703 - val_loss: 0.4640
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 263s 416ms/step - accuracy: 0.8692 - loss: 0.3219 - val_accuracy: 0.8426 - val_loss: 0.3656
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 261s 413ms/step - accuracy: 0.8865 - loss: 0.2833 - val_accuracy: 0.8606 - val_loss: 0.3350
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 205s 410ms/step - accuracy: 0.9053 - loss: 0.2381 - val_accuracy: 0.8776 - val_loss: 0.3212


# Model Evaluation

## Building a Predictive System

In [29]:
loss, accuracy = model.evaluate(X_test, Y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 43s 131ms/step - accuracy: 0.8807 - loss: 0.3105


In [31]:
print (loss)

0.30871331691741943


In [32]:
def predictive_system(review):

  # tokenize and pad the review

  sequence = tokenizer.texts_to_sequences([review])

  padded_sequence = pad_sequences(sequence, maxlen=200)

  prediction = model.predict(padded_sequence)

  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"

  return sentiment

In [33]:
predictive_system("This movie was fantastic and amazing")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step


'positive'

In [34]:
predictive_system("A trilling adventure with stunning visual")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step


'positive'

In [35]:
predictive_system("A visual masterpiece")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 553ms/step


'positive'

In [36]:
predictive_system("Overall long and slow")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 476ms/step


'negative'

In [37]:
model.save("model.h5")

In [38]:
import joblib
joblib.dump(tokenizer, 'tokenizer.pkl')

['tokenizer.pkl']